In [27]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import scipy.linalg
import matplotlib.pyplot as plt
import linear_shrinkage
import Util
from Util import evaluate_curret, get_invest_period
from os.path import join
from generating_MTP import generate_mat as generate_MTP2
import os
import time
import sklearn.covariance
import pickle
from collections import namedtuple, defaultdict
from subprocess import Popen
from multiprocessing import Pool

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
util_loaded_data = Util.load_data()
ret, ret_nonan, univ, tradeidx, dates = util_loaded_data

## For saving to pastRets (pastRet and KTcov) and outRets

In [35]:
assert os.path.exists('./pastRets')
assert os.path.exists('./outRets')

In [29]:
def save_past_information(T, N, util_loaded_data, save_KT = True):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRets = []
    KT_covs = []
    
    KT_time = 0
    for h in range(len(univ)):
        print(h, end = ' ')
        
        pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
        pastRetFileName = "pastRets/{}_{}_{}_pastRet.pkl".format(T, N, h)
        with open(pastRetFileName, 'wb') as f:
            pickle.dump(pastRet, f)
        pastRets.append(pastRet)
        
        if save_KT:
            #start = time.time()
            KT_cov = Util.kendall_cov(pastRet)
            KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
            with open(KT_cov_file_name, 'wb') as f:
                pickle.dump(KT_cov, f)
            KT_covs.append(KT_cov)
            end = time.time()
            #print(end-start)
    
    with open("pastRets/{}_{}_pastRets.pkl".format(T,N), 'wb') as f:
        pickle.dump(pastRets, f)

    if save_KT:
        with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

In [ ]:
#for old purposes
for T in [25, 50, 100, 200, 500, 1260]:
    for N in [25, 50, 100, 200, 500, 1000]:
        save_past_information(T, N, util_loaded_data, save_KT = False)

In [32]:
#for one-off
save_past_information(800,200, util_loaded_data, save_KT=False)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
#for saving KT covs in parallel
def save_KT_cov(h):
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    print(h)
    if h == 0:
        print("N={}, T={}".format(N,T))
    start = time.time()
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    pastRet = Util.get_past_period(h, T, N, univ, tradeidx, ret)
    KT_cov = Util.kendall_cov(pastRet)
    KT_cov_file_name =  "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
    with open(KT_cov_file_name, 'wb') as f:
        pickle.dump(KT_cov, f)
    end = time.time()
    print(end-start)

def consolidate_KT_cov(N,T):
    KT_covs = []
    for h in range(360):
        fname = "pastRets/{}_{}_{}_KTcov.pkl".format(T, N, h)
        with open(fname, 'rb') as f:
            cov = pickle.load(f)
            KT_covs.append(cov)
    with open("pastRets/{}_{}_KTcovs.pkl".format(T,N), 'wb') as f:
            pickle.dump(KT_covs, f)

for N, T in [(200,400)]:
    pool = Pool(8)
    pool.map(save_KT_cov,range(360))
    consolidate_KT_cov(N,T)

In [ ]:
def save_outrets(N, util_loaded_data, P=1):
    ret, ret_nonan, univ, tradeidx, dates = util_loaded_data
    all_outrets = []
    for h in range(360):
        outret = get_invest_period(h, P, N, univ, tradeidx, ret)
        all_outrets.append(outret)
    with open('outRets/{}_outRets.pkl'.format(N), 'wb') as f:
        pickle.dump(all_outrets, f)

for N in [25, 50, 100, 200, 500, 1000]:
    save_outrets(N, util_loaded_data)

## After this point is the algorithms and experiments, above was just generating data

In [4]:
from estimators import (MTP2_wrapper, MTP2_cov_wrapper,
                        LRPS_wrapper,
                        CLIME_wrapper, CLIME_cov_wrapper,
                        old_LS_wrapper, LS_wrapper, NLS_wrapper,
                        POET_wrapper, POET_5_wrapper,
                        glasso_wrapper,
                        get_AFM_estimator)

In [14]:
all_methods = { #all time estimates for N = 100
    'glasso': glasso_wrapper, #2 seconds
    'MTP2': MTP2_wrapper, #25 seconds
    #'MTP2_cov': MTP2_cov_wrapper,
    'CLIME': CLIME_wrapper,
    #'CLIME_cov': CLIME_cov_wrapper
    #'LS': LS_wrapper, #0.5 seconds
    #'old_LS_cov': old_LS_cov_wrapper,
    'old_LS': old_LS_wrapper,
    'NLS':  NLS_wrapper, #5 seconds
    'LRPS': LRPS_wrapper, #7 seconds
    'AFM_NL': get_AFM_estimator(5, 'NLS', tradeidx), #4 seconds
    'AFM_LS': get_AFM_estimator(5, 'LS', tradeidx), #0.5 seconds
    'POET': POET_wrapper,
    'POET_5': POET_5_wrapper,
    #'AFM_POET': None,
    #'equiweight': None,
    #'POET': POET_wrapper
}

### Time estimate

In [15]:
T = 1260
N = 100
with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
    pastRets = pickle.load(f)
timing_dict = {}
for method_name, m_func in all_methods.items():
    assert cov not in method_name, 'Timing is not supported for methods with cov'
    h = 0
    print(method_name, end = ' ')
    args = []
    start = time.time()
    if 'AFM' in method_name:
        args = [h]
    cov = m_func(pastRets[h], *args)
    end = time.time()
    print(end-start)
    timing_dict[method_name] = end-start

glasso 

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


2.037687063217163
MTP2 fd78b188-d8d9-4d9b-a387-866878a11eec
16.953660011291504
CLIME b'[1] "Loaded X"\n'
b'[1] 0.03984095\n'
b''
6.4462409019470215
old_LS 0.10021209716796875
NLS b'[1] "Loaded X"\n'
b'Estimating population eigenvalues...[1] "Finished with NLS"\n'
b''
3.3802690505981445
LRPS b'[1] "### Computing the path on the full dataset first ###"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 1.13453970662616 Sparsity: 0.00484848484848485 Rank of L: 3"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.568616817004261 Sparsity: 0.0121212121212121 Rank of L: 6"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.284983489508311 Sparsity: 0.042020202020202 Rank of L: 15"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.142830086736117 Sparsity: 0.145252525252525 Rank of L: 29"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.0715846160500178 Sparsity: 0.307878787878788 Rank of L: 36"\n'
b'[1] "Fitting with gamma= 0.15  and lambda= 0.0358772956883789 Sparsity: 0.430909090909091 Rank o

In [18]:
print(timing_dict.items())
print("Total hours: ", sum(timing_dict.values()))

dict_items([('glasso', 2.037687063217163), ('MTP2', 16.953660011291504), ('CLIME', 6.4462409019470215), ('old_LS', 0.10021209716796875), ('NLS', 3.3802690505981445), ('LRPS', 14.036257982254028), ('AFM_NL', 1.3425300121307373), ('AFM_LS', 0.15319514274597168), ('POET', 18.358459949493408), ('POET_5', 18.198095321655273)])
Total hours:  81.00660753250122


### Run versions

In [37]:
assert os.path.exists("./pickle")
assert os.path.exists('./run_info')

In [34]:
run_name = "first_server_run"
KT_or_not = False
N_T_list = [(100,50),(100,100),(100,200),(100,400),(100,1260),
            (200,100),(200,200),(200,400), (200,800), (200, 1260),
            (500,250), (500,500), (500,1000), (500,1260)]

print("Running with run_name={} and KT_or_not={}".format(run_name, KT_or_not))
print("All methods are: " + str(all_methods.keys()))
print("N, T list is: {}".format(N_T_list))
if input("Save information?") == "Y":
    with open("run_info/{}.info".format(run_name), 'wb') as f:
        run_info = {
            'run_name': run_name,
            'KT_or_not': KT_or_not,
            'N_T_list': N_T_list,
            'all_methods_keys': list(all_methods.keys())
        }
        pickle.dump(run_info, f)

for N, T in N_T_list:
    if not os.path.exists('pastRets/{}_{}_pastRets.pkl'.format(T,N)):
        print("pastRets doesn't exist for N,T={},{}".format(N,T))
        save_past_information(T,N, util_loaded_data, save_KT=False)
    if not os.path.exists('outRets/{}_outRets.pkl'.format(N)):
        print("outRets doesn't exist for N={}".format(N))
        save_outrets(N, util_loaded_data, P=1)
    
for N, T in N_T_list:
    with open('pastRets/{}_{}_pastRets.pkl'.format(T,N), 'rb') as f:
        pastRets = pickle.load(f)
        assert len(pastRets) == 360

    if os.path.isfile('pastRets/{}_{}_KTcovs.pkl'.format(T,N)):
        with open('pastRets/{}_{}_KTcovs.pkl'.format(T,N), 'rb') as f:
            pastCovs = pickle.load(f)
    else:
        print("pastCovs doesn't exist for T={} N={}".format(T,N))
        pastCovs = []

    with open('outRets/{}_outRets.pkl'.format(N), 'rb') as f:
        outRets = pickle.load(f)
        assert len(outRets) == 360

    print("Loaded all relevant information")

    def get_covs_all_methods(h):
        print("WORKING ON h={}".format(h))
        for method_name, method in all_methods.items():
            print('Starting on {}'.format(method_name))

            #USING PASTCOVS
            if 'cov' in method_name:
                args = []
                if 'LS' in method_name or 'CLIME' in method_name:
                    #args is number of samples
                    args = [T]
                cov = method(cov=pastCovs[h], *args)
            else:
            #USING PASTRETS
                args = []
                if 'AFM' in method_name:
                    args = [h]
                cov = method(pastRets[h], *args)

            with open('pickle/{}_{}_{}_{}_{}_{}_covEst.pkl'.format(T, N, h, method_name, KT_or_not, run_name), 'wb') as f:
                pickle.dump(cov, f)
    
    pool = Pool(8)
    pool.map(get_covs_all_methods, range(360))

Running with run_name=first_server_run and KT_or_not=False
All methods are: dict_keys(['glasso', 'MTP2', 'CLIME', 'old_LS', 'NLS', 'LRPS', 'AFM_NL', 'AFM_LS', 'POET', 'POET_5'])
N, T list is: [(100, 50), (100, 100), (100, 200), (100, 400), (100, 1260), (200, 100), (200, 200), (200, 400), (200, 800), (200, 1260), (500, 250), (500, 500), (500, 1000), (500, 1260)]
Save information?Y
Loaded all relevant information
WORKING ON h=24
WORKING ON h=36
WORKING ON h=48
WORKING ON h=60
WORKING ON h=0
WORKING ON h=12
WORKING ON h=72
Starting on glasso
Starting on glasso
Starting on glasso
Starting on glasso
WORKING ON h=84
Starting on glasso
Starting on glasso
Starting on glasso
Starting on glasso
WORKING ON h=96
Starting on glasso
WORKING ON h=108
Starting on glasso
WORKING ON h=120
Starting on glasso
WORKING ON h=132
WORKING ON h=144
Starting on glasso
Starting on glasso
WORKING ON h=156
WORKING ON h=168
Starting on glasso
Starting on glasso
WORKING ON h=180
Starting on glasso
WORKING ON h=192
St

Process ForkPoolWorker-93:
Process ForkPoolWorker-94:
Process ForkPoolWorker-95:
Process ForkPoolWorker-99:
Process ForkPoolWorker-100:
Process ForkPoolWorker-97:
Process ForkPoolWorker-96:
Process ForkPoolWorker-98:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/loca

KeyboardInterrupt: 

  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args